In [ ]:
pip install imgkit

In [1]:
import os
import pandas as pd
import dataframe_image as dfi

pd.set_option('display.max_rows',None, 'display.max_columns', None)

# CAPTURE UNPIVOT TABLE

In [9]:
df_quadran = pd.read_csv('../quadran/target.csv')
#target_pivot = df_quadran[['KABUPATEN/KOTA','Jumlah_DPT_2024','DPT_x_82%','TARGET_GP_2024','% TARGET_GP_2024']]
df_quadran.columns

Index(['PROPINSI', 'Kabupaten', 'KABUPATEN2', 'Kecamatan', 'Jumlah DPT 2024',
       ' DPT 2024 x 82% ', 'Target GP 2024', '% TARGET GP 2024',
       'Jumlah Kel/Desa', 'Jumah TPS', 'Prio', 'Kuadran'],
      dtype='object')

In [10]:
df_quadran.head()

,PROPINSI,Kabupaten,KABUPATEN2,Kecamatan,Jumlah DPT 2024,DPT 2024 x 82%,Target GP 2024,% TARGET GP 2024,Jumlah Kel/Desa,Jumah TPS,Prio,Kuadran
0,ACEH,ACEH SELATAN,ACEH SELATAN,BAKONGAN,3886,"3,186.5","1,578",49.51%,7.0,15.0,0,4
1,ACEH,ACEH SELATAN,ACEH SELATAN,KLUET UTARA,18230,"14,948.6","3,021",20.21%,21.0,71.0,0,4
2,ACEH,ACEH SELATAN,ACEH SELATAN,KLUET SELATAN,10686,"8,762.5","1,627",18.57%,17.0,44.0,0,4
3,ACEH,ACEH SELATAN,ACEH SELATAN,LABUHAN HAJI,9806,"8,040.9",956,11.89%,16.0,40.0,0,4
4,ACEH,ACEH SELATAN,ACEH SELATAN,MEUKEK,15479,"12,692.8","1,617",12.74%,23.0,64.0,0,4


In [ ]:
provinsi = [
    'BANTEN',
]
kabkot = df_quadran['KABUPATEN2'].unique().tolist()
kabkot

# FUNCTION 

In [14]:
# def (jenis, jumlah_dpt):

#     print(jenis)
#     print(jumlah_dpt)
    # jumlah_dpt = jumlah_dpt.replace(',','').astype('int64')
    # dpt_82 = dpt_82.replace(',','').astype('int64')
    # target_gp = target_gp.replace(',','').astype('int64')

    # data_dict = {
    #     '' : [jenis],
    #     'Jumlah DPT 2024 tes' : jumlah_dpt.sum(),
    #     'DPT x 82% tes' : dpt_82.sum(),
    #     'TARGET GP 2024 tes' : target_gp.sum()}

def total(tipe, data):
    x = data.copy()
    total = x[['Jumlah Kel/Desa','Jumlah TPS','DPT 2024','DPT 2024 x 82%','Target GP 2024']].sum().to_frame().T
    total['Kecamatan'] = tipe

    df = pd.concat([x, total]) 
    return df

def move_column(data):
    first_column = data.pop('Jumlah DPT 2024')
    two_column = data.pop(' DPT 2024 x 82% ')
    thre_column = data.pop('Target GP 2024')
    tps = data.pop('Jumah TPS')
    kel_des = data.pop('Jumlah Kel/Desa')
    target_gp_per = data.pop('% TARGET GP 2024')
    
    data.insert(0, 'Jumlah Kel/Desa', kel_des)
    data.insert(1, 'Jumlah TPS', tps)
    data.insert(2, 'DPT 2024', first_column)
    data.insert(3, 'DPT 2024 x 82%', two_column)
    data.insert(4, 'Target GP 2024', thre_column)
    data.insert(5, '% TARGET GP 2024', target_gp_per)

    data['Kuadran'].replace('.0','').astype('int64')
    
    return data

def format_color_groups(df):
    colors = ['','#FFCECD', '#FFF4D9','','#EBF1F1']
    x = df.copy()
    factors = list(x['Kuadran'].unique())

    for factor in factors:
        style = f'background-color: {colors[factor]}; color:black;'
        x.loc[x['Kuadran'] == factor, :] = style
    return x


In [ ]:
styled_df = move_column_prio.style.apply(format_color_groups, axis=None)
styled_df

In [7]:
df_quadran.columns

Index(['kode_pro_kemendagri_2022', 'kode_kab_kemendagri_2022', 'nama_provinsi',
       'Kabupaten/Kota', 'PKB', 'Gerindra', 'PDIP', 'Golkar', 'Nasdem',
       'Garuda', 'Berkarya', 'PKS', 'Perindo', 'PPP', 'PSI', 'PAN', 'Hanura',
       'Demokrat', 'PA', 'Partai SIRA', 'PD Aceh', 'PNA', 'PBB', 'PKPI'],
      dtype='object')

# TINGKAT KABUPATEN

In [74]:
df = pd.DataFrame([[1,2],[3,4],[5,6],[7,8]], index=["A", "B", "C", "D"])
green = [{'selector': 'th', 'props': 'background-color: green'}]
red = [{'selector': 'th', 'props': 'background-color: red'}]
df.style.set_table_styles({"A": green, "B": red, "C": green, "D": red}, axis=1)

,0,1
A,1,2
B,3,4
C,5,6
D,7,8


In [ ]:
data_null = []

for prop in range(0, len(provinsi)):
    for kab in range(0, len(kabkot)):

        try:
            prio = df_quadran[(df_quadran['PROPINSI'] == provinsi[prop]) & (df_quadran['KABUPATEN2'] == kabkot[kab]) & (df_quadran['Prio'] == 1)].pivot_table(index='Kecamatan', values=['Jumlah DPT 2024',' DPT 2024 x 82% ','Target GP 2024','Jumah TPS','Jumlah Kel/Desa','% TARGET GP 2024','Kuadran'], aggfunc='sum')
            non_prio = df_quadran[(df_quadran['PROPINSI'] == provinsi[prop]) & (df_quadran['KABUPATEN2'] == kabkot[kab]) & (df_quadran['Prio'] == 0)].pivot_table(index='Kecamatan', values=['Jumlah DPT 2024',' DPT 2024 x 82% ','Target GP 2024','Jumah TPS','Jumlah Kel/Desa','% TARGET GP 2024','Kuadran'], aggfunc='sum')
            prio_non_prio = df_quadran[(df_quadran['PROPINSI'] == provinsi[prop]) & (df_quadran['KABUPATEN2'] == kabkot[kab])].pivot_table(index='Kecamatan', values=['Jumlah DPT 2024',' DPT 2024 x 82% ','Target GP 2024','Jumah TPS','Jumlah Kel/Desa','% TARGET GP 2024'], aggfunc='sum')

            if prio.empty or non_prio.empty:
                pass
            else:
                move_column_prio = move_column(prio)
                move_column_non_prio = move_column(non_prio)
                
                move_column_prio.reset_index(inplace=True)
                move_column_non_prio.reset_index(inplace=True)

                move_column_prio.index += 1
                move_column_non_prio.index += 1

                #styled_prio = move_column_prio.style.apply(format_color_groups, axis=None)
                print(prio_non_prio)
                #styled_non_prio = move_column_non_prio.style.apply(format_color_groups, axis=None)
                
                # update_prio = total('Total Prioritas', move_column_prio)
                # update_non_prio = total('Total Non Prioritas', move_column_non_prio)
                
                # grand_total = prio_non_prio[['Jumlah Kel/Desa','Jumlah TPS','DPT 2024','DPT 2024 x 82%','Target GP 2024']].sum().to_frame().T
                # grand_total['Kecamatan'] = tipe
                # print(update_all)

                #result_prio = color_prio(update_prio)
                # result_non_prio = color_non_prio(update_non_prio)
                #result_total_prio = color_prio(total_prio)

                # outdir = f'{provinsi[prop]}/{kabkot[kab]}'
                # if not os.path.exists(outdir):
                #     os.mkdir(outdir)

                # dfi.export(styled_prio, f'../quadran/result/{provinsi[prop]}-{kabkot[kab]}-prioritas.jpg')
                # dfi.export(result_total_prio, f'result/{provinsi[prop]}-{kabkot[kab]}-grand_total_prioritas.jpg')
                # dfi.export(result_non_prio, f'result/{provinsi[prop]}-{kabkot[kab]}-non_prioritas.jpg')

        except KeyError as e:
            print(f"Error key not found {e}")
            break
        
    #print("Gagal Running Mase")
    # unpivot.insert(0, 'Jumlah DPT 2024', first_column)
    # unpivot.reset_index(inplace=True)

    #dfi.export(unpivot, f'result/{provinsi[prop]}.jpg')

In [341]:
import pandas as pd

# Dataframe awal
data = {
    'Kecamatan': ['LIMUN', 'PAUH', 'BATANG ASAI'],
    'Jumlah Kel/Desa': [16, 14, 23],
    'Jumah TPS': [61, 72, 67],
    'DPT 2024': [13.325, 17.699, 13.985]
}

df = pd.DataFrame(data)

# Menghitung total
total_nonprioritas = df[['Jumlah Kel/Desa', 'Jumah TPS', 'DPT 2024']].sum().to_frame().T
total_nonprioritas['Kecamatan'] = 'Total Nonprioritas'

# Menggabungkan dataframe
df = pd.concat([df, total_nonprioritas], ignore_index=True)

outdir = 'dir'
if not os.path.exists(outdir):
    os.mkdir(outdir)

df.to_csv(f'{outdir}/asw.csv')


# CAPTURE PILEG AND PILPRES WITH COLOR

In [96]:
file_dpr_ri = '../request_mas_zima/capture_prioritas/0112023_2019_pers_suara_dpr_ri_kec_coba.csv'
file_pilpres = '../request_mas_zima/capture_prioritas/0111013_2019_suara_pilpres_kec.csv'

pileg = pd.read_csv(file_dpr_ri)
pilpres = pd.read_csv(file_pilpres)

pilpres.head()

,kode_pro_kemendagri_2022,kode_kab_kemendagri_2022,kode_kec_kemendagri_2022,nama_pro,nama_kab,Kecamatan,gabungan,JW-MA,PS-SU,JW-MA(%),PS-SU(%)
0,11,1101,110101,ACEH,ACEH SELATAN,BAKONGAN,ACEH-ACEH SELATAN-BAKONGAN,376,2742,0.120590,0.879410
1,11,1101,110102,ACEH,ACEH SELATAN,KLUET UTARA,ACEH-ACEH SELATAN-KLUET UTARA,1517,12623,0.107284,0.892716
2,11,1101,110103,ACEH,ACEH SELATAN,KLUET SELATAN,ACEH-ACEH SELATAN-KLUET SELATAN,910,7431,0.109100,0.890900
3,11,1101,110104,ACEH,ACEH SELATAN,LABUHAN HAJI,ACEH-ACEH SELATAN-LABUHAN HAJI,639,7051,0.083095,0.916905
4,11,1101,110105,ACEH,ACEH SELATAN,MEUKEK,ACEH-ACEH SELATAN-MEUKEK,779,11754,0.062156,0.937844


In [27]:
pileg.head()

,kode_pro_kemendagri_2022,kode_kab_kemendagri_2022,kode_kec_kemendagri_2022,nama_pro,nama_kab,Kecamatan,gabungan,PKB,Gerindra,PDIP,Golkar,Nasdem,Garuda,Berkarya,PKS,Perindo,PPP,PSI,PAN,Hanura,Demokrat,PA,Partai SIRA,PD Aceh,PNA,PBB,PKPI
0,11,1101,110101,ACEH,ACEH SELATAN,BAKONGAN,ACEH-ACEH SELATAN-BAKONGAN,0.017017,0.024358,0.002669,0.100434,0.008342,0.003003,0.007007,0.015682,0.004004,0.671004,0.005005,0.022689,0.005672,0.094761,0,0,0,0,0.007007,0.011345
1,11,1101,110102,ACEH,ACEH SELATAN,KLUET UTARA,ACEH-ACEH SELATAN-KLUET UTARA,0.078010,0.044892,0.008735,0.165515,0.052032,0.003570,0.024003,0.033954,0.006760,0.157463,0.011394,0.251348,0.013217,0.105203,0,0,0,0,0.033726,0.010179
2,11,1101,110103,ACEH,ACEH SELATAN,KLUET SELATAN,ACEH-ACEH SELATAN-KLUET SELATAN,0.101213,0.031303,0.013173,0.168254,0.078518,0.002087,0.020608,0.053085,0.004304,0.154037,0.016956,0.209600,0.006652,0.111778,0,0,0,0,0.011478,0.016956
3,11,1101,110104,ACEH,ACEH SELATAN,LABUHAN HAJI,ACEH-ACEH SELATAN-LABUHAN HAJI,0.056653,0.022938,0.004422,0.139975,0.081111,0.003869,0.011469,0.070609,0.004560,0.025287,0.004974,0.120630,0.011331,0.430980,0,0,0,0,0.005113,0.006080
4,11,1101,110105,ACEH,ACEH SELATAN,MEUKEK,ACEH-ACEH SELATAN-MEUKEK,0.150560,0.027467,0.003792,0.124480,0.082493,0.006474,0.020253,0.059188,0.010820,0.050865,0.015167,0.202904,0.005826,0.191991,0,0,0,0,0.021733,0.025987


In [97]:
columns_pileg = pileg.columns.to_list()
columns_pilpres = pilpres.columns.to_list()
provinsi = pileg['nama_pro'].unique().tolist()
kabupaten = pileg['nama_kab'].unique().tolist()

#provinsi = ['BANTEN']
columns_pilpres

['kode_pro_kemendagri_2022',
 'kode_kab_kemendagri_2022',
 'kode_kec_kemendagri_2022',
 'nama_pro',
 'nama_kab',
 'Kecamatan',
 'gabungan',
 'JW-MA',
 'PS-SU',
 'JW-MA(%)',
 'PS-SU(%)']

In [98]:
#pileg['Max Value'] = pileg[columns_pileg[7:]].max(axis=1)
pileg['pemenang'] = pileg[columns_pileg[7:]].idxmax(axis=1)

pilpres['Max Value'] = pilpres[columns_pilpres[9:]].max(axis=1)
pilpres['pemenang'] = pilpres[columns_pilpres[9:]].idxmax(axis=1)

columns_pileg = pileg.columns.to_list()

pilpres.head(5)

C:\Users\User\AppData\Local\Temp\ipykernel_22084\3080149201.py:5: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  pilpres['pemenang'] = pilpres[columns_pilpres[9:]].idxmax(axis=1)


,kode_pro_kemendagri_2022,kode_kab_kemendagri_2022,kode_kec_kemendagri_2022,nama_pro,nama_kab,Kecamatan,gabungan,JW-MA,PS-SU,JW-MA(%),PS-SU(%),Max Value,pemenang
0,11,1101,110101,ACEH,ACEH SELATAN,BAKONGAN,ACEH-ACEH SELATAN-BAKONGAN,376,2742,0.120590,0.879410,0.879410,PS-SU(%)
1,11,1101,110102,ACEH,ACEH SELATAN,KLUET UTARA,ACEH-ACEH SELATAN-KLUET UTARA,1517,12623,0.107284,0.892716,0.892716,PS-SU(%)
2,11,1101,110103,ACEH,ACEH SELATAN,KLUET SELATAN,ACEH-ACEH SELATAN-KLUET SELATAN,910,7431,0.109100,0.890900,0.890900,PS-SU(%)
3,11,1101,110104,ACEH,ACEH SELATAN,LABUHAN HAJI,ACEH-ACEH SELATAN-LABUHAN HAJI,639,7051,0.083095,0.916905,0.916905,PS-SU(%)
4,11,1101,110105,ACEH,ACEH SELATAN,MEUKEK,ACEH-ACEH SELATAN-MEUKEK,779,11754,0.062156,0.937844,0.937844,PS-SU(%)


In [30]:
pileg.to_excel('cek.xlsx')

# FUNCTION

In [99]:
warna_partai = {
    'PDIP': 'background-color: #db2016; color: white; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Kabupaten/Kota': 'background-color: #db2016; color: white; font-weight: 700; text-align: center; font-family: "Montserrat", sans-serif;',
    'Kota / Kab': 'background-color: #db2016; color: white; font-weight: 700; text-align: center; font-family: "Montserrat", sans-serif;',
    'Kecamatan': 'background-color: #db2016; color: white; font-weight: 700; text-align: center; font-family: "Montserrat", sans-serif;',
    'JW-MA': 'background-color: #db2016; color: white; font-weight: 700; text-align: center; font-family: "Montserrat", sans-serif;',
    'PS-SU': 'background-color: #db2016; color: white; font-weight: 700; text-align: center; font-family: "Montserrat", sans-serif;',
    'JW-MA(%)': 'background-color: #db2016; color: white; font-weight: 700; text-align: center; font-family: "Montserrat", sans-serif;',
    'PS-SU(%)': 'background-color: #db2016; color: white; font-weight: 700; text-align: center; font-family: "Montserrat", sans-serif;',
    'Golkar': 'background-color: #ffff00 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Gerindra': 'background-color: #b79164 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Nasdem': 'background-color: #99d9ea ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PKB': 'background-color: #008000 ; color: white; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Demokrat': 'background-color: #2643a3 ; color: white; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PAN': 'background-color: #0033ff ; color: white; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PKS': 'background-color: #000000 ; color: white; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PPP': 'background-color: #00b300 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Perindo': 'background-color: #7583a9 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Berkarya': 'background-color: #eeee22 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Hanura': 'background-color: #ed9a11 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PSI': 'background-color: #ff00ff ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PBB': 'background-color: #00ff00 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Garuda': 'background-color: #e6b333 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PKPI': 'background-color: #ff0000 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PD Aceh': 'background-color: #118dff ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PA': 'background-color: #fa8072 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Partai SIRA': 'background-color: #00beff ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PNA': 'background-color: #fbceb1 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'pemenang':'',
    'pilpres_jwma':''}

def select_col(source):
    
    filter = source.columns.to_list()
    filter.remove('Max Value')

    df1 =  pd.DataFrame(index=source.index, columns=source.columns)

    for col in filter[1:]:
        compare = source[col].eq(source['Max Value'])
        df1.loc[compare, col] = warna_partai[col]

    return df1

def table_style(source):
    filter = source.columns.to_list()
    filter.remove('Max Value')

    for col in filter:

        font_css = '"Montserrat", sans-serif;'
        header_styles = {
            col : [{'selector': 'th',  'props': warna_partai[col]}],
        }
        
        source = source.set_table_styles(header_styles, overwrite=False)
        
        format_dict = {col: '{:.1%}' for col in filter[1:]}
        convert = source.format(format_dict)
        
        convert.set_properties(**{'text-align':'center','font-family': font_css})

    return convert

def table_style_pilpres(source):
    filter = source.columns.to_list()
    filter.remove('Max Value')

    for col in filter:
        header_styles = {
            col : [{'selector': 'th',  'props': warna_partai[col]}],
        }
        
        source = source.set_table_styles(header_styles, overwrite=False)
        source.set_properties(**{'text-align':'center','font-family': '"Montserrat", sans-serif;'})

    return source

def move_column_pilpres(data):
    first_column = data.pop('PS-SU')
    two_column = data.pop('PS-SU(%)')	
    
    data.insert(2, 'PS-SU', first_column)
    data.insert(4, 'PS-SU(%)', two_column)

    return data

def apply_style(row):
    background = {
        'A' : 'background-color: #eaa00a; color: white; font-weight: bold;',
        'B' : 'background-color: #ffba14; color: white; font-weight: bold;',
        'C' : 'background-color: #fed12b; color: white; font-weight: bold;', 
        'D' : 'background-color: #f9d97d; color: white; font-weight: bold;', 
        'F' : 'background-color: #ff8080; color: white; font-weight: bold;', 
        'G' : 'background-color: #ff6666; color: white; font-weight: bold;', 
        'H' : 'background-color: #ff5550; color: white; font-weight: bold;',
        'I' : 'background-color: #ff2929; color: white; font-weight: bold;'}

    styles = {col: '' for col in move_pilpres.columns}
    for col in move_pilpres.columns:
        if row['pemenang'] == col and col in ['JW-MA(%)', 'PS-SU(%)']:
            styles[col] = background[row['pilpres_jwma']]
    return pd.Series(styles, index=move_pilpres.columns)  

def classify_pilpres(jwma):
    if 0 <= jwma < 0.15:
        return 'A'
    elif 0.15 <= jwma < 0.30:
        return 'B'
    elif 0.30 <= jwma < 0.45:
        return 'C'
    elif 0.45 <= jwma < 0.50:
        return 'D'
    elif 0.50 <= jwma < 0.55:
        return 'F'
    elif 0.55 <= jwma < 0.70:
        return 'G'
    elif 0.70 <= jwma < 0.85:
        return 'H'
    else:
        return 'I'

In [72]:
styles = {
    'Kecamatan' : [{'selector': 'td', 'props': [('text-align', 'left')]}]
}
tes = []
pilpres['pilpres_jwma'] = pilpres['JW-MA(%)'].apply(classify_pilpres)
columns_pilpres = pilpres.columns.to_list()

def unpivot_data(source_pil, source_pilpres, index, provinsi, kabkot):
    unpivot_pileg = source_pil[(source_pil['nama_pro'] == provinsi) & (source_pil['nama_kab'] == kabkot)].pivot_table(index=index, values=columns_pileg[7:], aggfunc='sum')
    unpivot_pilpres = source_pilpres[(source_pilpres['nama_pro'] == provinsi) & (source_pilpres['nama_kab'] == kabkot)].pivot_table(index=index, values=columns_pilpres[7:], aggfunc='sum')

    if unpivot_pileg.empty or unpivot_pilpres.empty:
        pass
    else:
        #pilpres
        unpivot_pilpres.reset_index(inplace=True)
        move_pilpres = move_column_pilpres(unpivot_pilpres)
        color_pilpres = unpivot_pilpres.style.hide().apply(apply_style, axis=1)
        hide_col = color_pilpres.hide(['Max Value','pemenang','pilpres_jwma'], axis=1)
        style_pilpres = table_style_pilpres(hide_col)
        format_dict = {'JW-MA(%)': '{:.1%}', 'PS-SU(%)': '{:.1%}', 'JW-MA':'{:,}', 'PS-SU':'{:,}'}
        convert_pilpres = style_pilpres.format(format_dict)
        convert_pilpres = convert_pilpres.set_table_styles(styles, overwrite=False)
        #coloring_pileg = move_pilpres.style.apply(table_style, axis=None)

        #pileg
        # unpivot_pileg.reset_index(inplace=True)
        # unpivot_pileg.index = unpivot_pileg.index + 1
        # pemenang_pileg = unpivot_pileg['pemenang'].unique().tolist()
        # pemenang_pileg.insert(0, 'Kecamatan')
        # filter_pileg = unpivot_pileg[pemenang_pileg]
        # filter_pileg['Max Value'] = unpivot_pileg[pemenang_pileg[1:]].max(axis=1)

        # coloring_pileg = filter_pileg.style.hide().apply(select_col, axis=None)
        # convert_pileg = coloring_pileg.set_table_styles(styles, overwrite=False)
        # update_pileg = convert_pileg.hide(['Max Value'], axis=1)
        # update_pileg = table_style(update_pileg)
        
        presiden = f'../request_mas_zima/capture_prioritas/capture_pilpres/{pro}'
        dpr_ri = f'../request_mas_zima/capture_prioritas/capture_dpr_ri/{pro}'
        bikin = f'dir/{pro}'

        if not os.path.exists(presiden):
            os.makedirs(presiden)

        #dfi.export(update_pileg, f'{dpr_ri}/KABUPATEN-{kab}.jpg')
        # dfi.export(convert_pilpres, f'{presiden}/KABUPATEN-{kab}.jpg')

In [73]:
for pro in provinsi:
    for kab in kabupaten:
        unpivot_data(pileg, pilpres, 'Kecamatan', pro, kab)

KeyError: 'PS-SU'

# TINGKAT KABUPATEN

In [100]:
styles = {
    'Kecamatan' : [{'selector': 'td', 'props': [('text-align', 'left')]}]
}
tes = []
pilpres['pilpres_jwma'] = pilpres['JW-MA(%)'].apply(classify_pilpres)
columns_pilpres = pilpres.columns.to_list()

for pro in provinsi:
    for kab in kabupaten:
        unpivot_pileg = pileg[(pileg['nama_pro'] == pro) & (pileg['nama_kab'] == kab)].pivot_table(index='Kecamatan', values=columns_pileg[7:], aggfunc='sum')
        unpivot_pilpres = pilpres[(pilpres['nama_pro'] == pro) & (pilpres['nama_kab'] == kab)].pivot_table(index='Kecamatan', values=columns_pilpres[7:], aggfunc='sum')

        if unpivot_pileg.empty or unpivot_pilpres.empty:
            pass
        else:
            #pilpres
            unpivot_pilpres.reset_index(inplace=True)
            move_pilpres = move_column_pilpres(unpivot_pilpres)
            color_pilpres = move_pilpres.style.hide().apply(apply_style, axis=1)
            hide_col = color_pilpres.hide(['Max Value','pemenang','pilpres_jwma'], axis=1)
            style_pilpres = table_style_pilpres(hide_col)
            format_dict = {'JW-MA(%)': '{:.1%}', 'PS-SU(%)': '{:.1%}', 'JW-MA':'{:,}', 'PS-SU':'{:,}'}
            convert_pilpres = style_pilpres.format(format_dict)
            convert_pilpres = convert_pilpres.set_table_styles(styles, overwrite=False)
            #coloring_pileg = move_pilpres.style.apply(table_style, axis=None)

            #pileg
            # unpivot_pileg.reset_index(inplace=True)
            # unpivot_pileg.index = unpivot_pileg.index + 1
            # pemenang_pileg = unpivot_pileg['pemenang'].unique().tolist()
            # pemenang_pileg.insert(0, 'Kecamatan')
            # filter_pileg = unpivot_pileg[pemenang_pileg]
            # filter_pileg['Max Value'] = unpivot_pileg[pemenang_pileg[1:]].max(axis=1)

            # coloring_pileg = filter_pileg.style.hide().apply(select_col, axis=None)
            # convert_pileg = coloring_pileg.set_table_styles(styles, overwrite=False)
            # update_pileg = convert_pileg.hide(['Max Value'], axis=1)
            # update_pileg = table_style(update_pileg)
            
            presiden = f'../request_mas_zima/capture_prioritas/capture_pilpres/{pro}'
            dpr_ri = f'../request_mas_zima/capture_prioritas/capture_dpr_ri/{pro}'

            if not os.path.exists(dpr_ri):
                os.makedirs(dpr_ri)

            #dfi.export(update_pileg, f'{dpr_ri}/KABUPATEN-{kab}.jpg')
            dfi.export(convert_pilpres, f'{presiden}/KABUPATEN-{kab}.jpg')

In [17]:
tes

KeyboardInterrupt: 

In [ ]:
result = partai.style.apply(select_col, axis=None)
result = result.hide(['Max Value'], axis=1)
result

# #add color per column
header_styles = {
    'nama_provinsi': [{'selector': 'th','props': [('background-color', f'{pdip}'),('color','white'),('font-family', 'Montserrat'),('text-align','center')]}],
    'Kabupaten/Kota': [{'selector': 'th','props': [('background-color', f'{pdip}'),('color','white'),('font-family', 'Montserrat'),('text-align','center')]}],
    'Gerindra': [{'selector': 'th','props': [('background-color', f'{gerindra}'),('color','black'),('font-family', 'Montserrat'),('text-align','center')]}],
    'Golkar': [{'selector': 'th','props': [('background-color',f'{golkar}'),('color','black'),('font-family', 'Montserrat'),('text-align','center')]}],
    'PDIP': [{'selector': 'th','props': [('background-color',f'{pdip}'),('color','white'),('font-family', 'Montserrat'),('text-align','center')]}],
    'PKS': [{'selector': 'th','props': f'background-color:{pks}; color:white; font-family: Montserrat; text-align: center;'}]
}

result.set_table_styles(header_styles, overwrite=False)
convert = result.format({'Gerindra': '{:.1%}','PDIP': '{:.1%}','Golkar': '{:.1%}','PKS': '{:.1%}'})

font_css = "font-family: 'Montserrat';"
convert.set_properties(**{'text-align': 'center', 'font-family': font_css})